In [0]:
# display(dbutils.fs.mounts())

In [0]:
#%fs
#ls /mnt/formula1dlriki/raw

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType , TimestampType

In [0]:
races_schema = StructType(fields=[StructField("raceId",IntegerType(),False),
                                 StructField("year",IntegerType(),True),
                                 StructField("round",IntegerType(),True),
                                 StructField("circuitId",IntegerType(),True),
                                 StructField("name",StringType(),True),
                                 StructField("date",StringType(),True),
                                 StructField("time",StringType(),True),
                                 StructField("url",StringType(),True)])

In [0]:
races_df = spark.read.schema(races_schema).csv(f"{raw_folder_path}/{v_file_date}/races.csv",header=True)

In [0]:
# display(races_df)

In [0]:
from pyspark.sql.functions  import col, lit

In [0]:
races_selected_df = races_df.select(col("raceId").alias("race_id"), col("year").alias("race_year"),col("round"), \
                                    col("circuitId").alias("circuit_id"), col("name"), col("date"),col("time"))

In [0]:
from pyspark.sql.functions import current_timestamp, to_timestamp, concat

In [0]:
# display(races_selected_df)

In [0]:
selected_races_df = races_selected_df.withColumn('race_timestamp', to_timestamp(concat(col('date'),lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss')) \
    .withColumn("ingestion_date",current_timestamp()) \
    .withColumn("data_source", lit(v_data_source)) \
    .withColumn("file_date", lit(v_file_date))

In [0]:
# selected_races_df.printSchema()

In [0]:
final_races_df = selected_races_df.drop('date').drop('time')

In [0]:
# display(final_races_df)

In [0]:
# final_races_df.write.mode("overwrite").partitionBy('race_year').format("parquet").saveAsTable("f1_processed.races")
final_races_df.write.mode("overwrite").partitionBy('race_year').format("delta").saveAsTable("f1_processed.races")

In [0]:
#%fs
#ls /mnt/formula1dlriki/processed/races

In [0]:
# df = spark.read.parquet("/mnt/formula1dlriki/processed/races")

In [0]:
# display(df)

In [0]:
dbutils.notebook.exit("Success")

Success

In [0]:
v_file_date